In [3]:
import os
#os.system('pip install scanpy')
import scanpy
import pandas as pd
import numpy as np
import scanpy
from sklearn.neighbors import NearestNeighbors

In [4]:
adata = scanpy.read_h5ad('data/sc_training.h5ad')
adata.obs

,gRNA_maxID,state,condition,lane
053l1_AAACCTGAGATGTCGG-1,ONE-NON-GENE-SITE-7,terminal exhausted,Unperturbed,lane1
053l1_AAACCTGAGCAACGGT-1,Tox2-3,effector,Tox2,lane1
053l1_AAACCTGAGTACGACG-1,Tpt1-2,effector,Tpt1,lane1
053l1_AAACCTGAGTCGTTTG-1,Tox2-3,terminal exhausted,Tox2,lane1
053l1_AAACCTGAGTGAAGAG-1,Tcf7-2,effector,Tcf7,lane1
...,...,...,...,...
053l4_TTTGTCATCAGGTTCA-1,Tox2-3,other,Tox2,lane4
053l4_TTTGTCATCAGTGTTG-1,Dvl2-3,cycling,Dvl2,lane4
053l4_TTTGTCATCCTCGCAT-1,Zeb2-2,cycling,Zeb2,lane4
053l4_TTTGTCATCTTCAACT-1,Sox4-3,cycling,Sox4,lane4


In [5]:
counts = adata.obs.condition.value_counts()
counts

Unperturbed    4978
Tox2           4333
Arid5b         2402
Dvl2           2173
Irf2           1155
               ... 
Batf              6
Klf2              5
Tbx21             2
Rps6              2
Eef2              1
Name: condition, Length: 67, dtype: int64

In [6]:
good_genes = counts[counts > 30].index
good_genes

CategoricalIndex(['Unperturbed', 'Tox2', 'Arid5b', 'Dvl2', 'Irf2', 'Il12rb1',
                  'Zeb2', 'Crem', 'Hif1a', 'Nr4a2', 'Arid4b', 'Dvl1', 'Nr4a3',
                  'Sp100', 'Ikzf3', 'Satb1', 'Prdm1', 'Hmgb1', 'Nr3c1',
                  'Litaf', 'Runx2', 'Sox4', 'Zfp292', 'Tcf7', 'Fzd1', 'Gsk3b',
                  'Egr1', 'Fzd6', 'Ldhb', 'Il12rb2', 'Nr4a1', 'Foxp1', 'Atf2',
                  'Fzd3', 'Sub1', 'Id2', 'Lef1', 'Irf9', 'Hmgb2', 'Rela',
                  'Dvl3', 'Foxo1', 'Tcf3', 'Elf1', 'Lrp1', 'Oxnad1', 'Foxm1',
                  'Dkk3', 'Id3', 'P2rx7', 'Ctnnb1', 'Tox', 'Sp140', 'Myb'],
                 categories=['Arid4b', 'Arid5b', 'Atf2', 'Batf', ..., 'Unperturbed', 'Yy1', 'Zeb2', 'Zfp292'], ordered=False, dtype='category')

In [9]:
nn_genes = set(good_genes) - {'Fzd1', 'P2rx7', 'Unperturbed'}
nn_genes = list(nn_genes) + ['Aqr', 'Bach2', 'Bhlhe40', 'Ets1', 'Fosb', 'Mafk', 'Stat3']
nn_genes

['Id2',
 'Dvl2',
 'Zeb2',
 'Litaf',
 'Gsk3b',
 'Sox4',
 'Foxp1',
 'Lrp1',
 'Nr4a2',
 'Dkk3',
 'Nr4a1',
 'Fzd3',
 'Satb1',
 'Sp100',
 'Ikzf3',
 'Zfp292',
 'Irf9',
 'Sub1',
 'Ldhb',
 'Nr3c1',
 'Oxnad1',
 'Sp140',
 'Lef1',
 'Hmgb1',
 'Dvl1',
 'Hif1a',
 'Foxo1',
 'Arid4b',
 'Runx2',
 'Nr4a3',
 'Elf1',
 'Fzd6',
 'Tcf7',
 'Prdm1',
 'Il12rb2',
 'Il12rb1',
 'Crem',
 'Tox2',
 'Dvl3',
 'Myb',
 'Foxm1',
 'Tox',
 'Hmgb2',
 'Id3',
 'Ctnnb1',
 'Irf2',
 'Tcf3',
 'Rela',
 'Egr1',
 'Arid5b',
 'Atf2',
 'Aqr',
 'Bach2',
 'Bhlhe40',
 'Ets1',
 'Fosb',
 'Mafk',
 'Stat3']

In [10]:
mean_data = {}
targets = ['progenitor', 'effector', 'terminal exhausted', 'cycling', 'other']
for i, gene in enumerate(nn_genes[:-7]):
    for target in targets:
        sh = adata[adata.obs.condition == gene].shape
        mean_ = 0
        if len(sh):
            sh1 = adata[(adata.obs.condition == gene) & (adata.obs.state == target)].shape
            if len(sh1):
                mean_ = sh1[0] / sh[0]
        mean_data[(i, target)] = mean_

mean_data

{(0, 'progenitor'): 0.27350427350427353,
 (0, 'effector'): 0.20512820512820512,
 (0, 'terminal exhausted'): 0.21367521367521367,
 (0, 'cycling'): 0.2905982905982906,
 (0, 'other'): 0.017094017094017096,
 (1, 'progenitor'): 0.024390243902439025,
 (1, 'effector'): 0.1155085135757018,
 (1, 'terminal exhausted'): 0.3041877588587207,
 (1, 'cycling'): 0.5494707777266452,
 (1, 'other'): 0.006442705936493327,
 (2, 'progenitor'): 0.017482517482517484,
 (2, 'effector'): 0.11538461538461539,
 (2, 'terminal exhausted'): 0.28205128205128205,
 (2, 'cycling'): 0.4393939393939394,
 (2, 'other'): 0.1456876456876457,
 (3, 'progenitor'): 0.07162534435261708,
 (3, 'effector'): 0.3526170798898072,
 (3, 'terminal exhausted'): 0.2369146005509642,
 (3, 'cycling'): 0.3278236914600551,
 (3, 'other'): 0.011019283746556474,
 (4, 'progenitor'): 0.2978723404255319,
 (4, 'effector'): 0.11063829787234042,
 (4, 'terminal exhausted'): 0.251063829787234,
 (4, 'cycling'): 0.31063829787234043,
 (4, 'other'): 0.02978723404

In [41]:
keys = []
for gene in nn_genes:
    keys += [np.where(adata.var.index == gene)[0][0]]

In [13]:
clf = NearestNeighbors(n_neighbors=5, p=1)
clf.fit(adata.X.T[keys[:-7]])

NearestNeighbors(p=1)

In [40]:
clf.kneighbors(adata.X.T[keys[-7:]])[1]

array([[11,  9,  7,  5, 31],
       [11,  9,  7,  5, 31],
       [ 3,  8,  0, 17, 25],
       [13, 30, 25, 41, 28],
       [11,  7,  9,  5, 31],
       [11,  9,  7,  5, 31],
       [ 3,  0,  8, 17, 25]])

In [17]:
answer = []
for ind in clf.kneighbors(adata.X.T[keys[-7:]])[1]:
    tmp_answer = [np.mean([mean_data[(i, target)] for i in ind]) for target in targets]
    answer += [tmp_answer]

pd.DataFrame(answer)

,0,1,2,3,4
0,0.178535,0.189973,0.275881,0.342471,0.013140
1,0.178535,0.189973,0.275881,0.342471,0.013140
2,0.137249,0.189595,0.278757,0.381684,0.012715
3,0.108088,0.292190,0.257613,0.326842,0.015268
4,0.178535,0.189973,0.275881,0.342471,0.013140
5,0.178535,0.189973,0.275881,0.342471,0.013140
6,0.137249,0.189595,0.278757,0.381684,0.012715


In [15]:
predict_df = pd.DataFrame(answer[:3], columns=['a_i', 'b_i', 'c_i', 'd_i', 'e_i'])
predict_df['gene'] = ['Aqr', 'Bach2', 'Bhlhe40']
predict_df = predict_df[['gene', 'a_i', 'b_i', 'c_i', 'd_i', 'e_i']]
predict_df.to_csv('validation_output.csv', index=None)
predict_df

,gene,a_i,b_i,c_i,d_i,e_i
0,Aqr,0.178535,0.189973,0.275881,0.342471,0.013140
1,Bach2,0.178535,0.189973,0.275881,0.342471,0.013140
2,Bhlhe40,0.137249,0.189595,0.278757,0.381684,0.012715


In [16]:
predict_df = pd.DataFrame(answer[3:], columns=['a_i', 'b_i', 'c_i', 'd_i', 'e_i'])
predict_df['gene'] = ['Ets1', 'Fosb', 'Mafk', 'Stat3']
predict_df = predict_df[['gene', 'a_i', 'b_i', 'c_i', 'd_i', 'e_i']]
predict_df.to_csv('test_output.csv', index=None)
predict_df

,gene,a_i,b_i,c_i,d_i,e_i
0,Ets1,0.108088,0.292190,0.257613,0.326842,0.015268
1,Fosb,0.178535,0.189973,0.275881,0.342471,0.013140
2,Mafk,0.178535,0.189973,0.275881,0.342471,0.013140
3,Stat3,0.137249,0.189595,0.278757,0.381684,0.012715
